In [1]:
%config IPCompleter.greedy=True
import pandas as pd
import numpy as np

#Changing the location of this to maintain portability
kick_data = pd.read_csv('/Users/chandlervaughn/Dropbox/4. Chandler/Development/Machine Learning/Lab1/kick.csv')
#kick_data=pd.read_csv("https://www.dropbox.com/s/q2h8uypvc0et5vj/kick.csv?dl=1")

FileNotFoundError: [Errno 2] File b'/Users/chandlervaughn/Dropbox/4. Chandler/Development/Machine Learning/Lab1/kick.csv' does not exist: b'/Users/chandlervaughn/Dropbox/4. Chandler/Development/Machine Learning/Lab1/kick.csv'

### 1.3.2 Data Cleanup and Imputation

In [2]:
#drop columns that have too little information to be useful
kick_data.columns = kick_data.columns.str.replace(' ', '')

kick_data = kick_data.drop(['PRIMEUNIT',
                            'AUCGUART', 
                            'PurchDate', 
                            'VehYear',
                            'VNZIP1', 
                            'WheelTypeID', 
                            'Nationality', 
                            #'model_short', 
                            'TopThreeAmericanName', 
                            'Trim', 
                            #'Transmission', 
                            #'Auction', 
                            #'Luxury', 
                            #'Make', 
                            'SubModel',
                            'MMRAcquisitionAuctionCleanPrice',
                            'MMRAcquisitionRetailCleanPrice',
                            'MMRCurrentAuctionAveragePrice',
                            'MMRCurrentAuctionCleanPrice',
                            'MMRCurrentRetailAveragePrice',
                            'MMRCurrentRetailCleanPrice',],axis=1)

# Feature Engineering

In [3]:
# Add Cylinder 
temp=kick_data.Model.fillna("0")
kick_data['Cylinder'] = pd.np.where(temp.str.contains("V6"),"V6",
                   pd.np.where(temp.str.contains("6C"), "V6",
                   pd.np.where(temp.str.contains("4C"), "V4", 
                   pd.np.where(temp.str.contains("8C"), "V8",            
                   pd.np.where(temp.str.contains("V4"), "V4",
                   pd.np.where(temp.str.contains("V8"), "V8", "Unknown"))))))  

# Add Axle
temp1=kick_data.Model.fillna("0")
kick_data['Axle'] = pd.np.where(temp1.str.contains("2WD"),"2WD",
                     pd.np.where(temp1.str.contains("4WD"), "4WD",
                     pd.np.where(temp1.str.contains("FWD"), "FWD",
                     pd.np.where(temp1.str.contains("AWD"), "AWD", "Unknown"))))

# Add Mileage
kick_data.VehOdo=kick_data.VehOdo.fillna("0")
kick_data.VehicleAge=kick_data.VehicleAge.fillna("1")
col1 = 'VehOdo'
col2 = 'VehicleAge'
conditions  = [ (kick_data[col1]/kick_data[col2]) < 10000, ((kick_data[col1]/kick_data[col2]) < 13867) & ((kick_data[col1]/kick_data[col2]) > 10000), (kick_data[col1]/kick_data[col2]) > 13867 ]
choices     = [ "LOW", 'GOOD', 'HIGH' ]
kick_data["Mileage"] = np.select(conditions, choices, default=np.nan)

In [4]:
#should be floats
continuous_features = ['VehOdo', 
                       'MMRAcquisitionAuctionAveragePrice',
                       'MMRAcquisitionRetailAveragePrice',
                       'VehBCost',
                       'WarrantyCost']

#should be int
ordinal_features = ['IsBadBuy', 
                    'VehicleAge', 
                    'BYRNO', 
                    'IsOnlineSale']


#should be categorical - Add Luxury, Axle, and Mileage
categorical_features = ['Auction',
                        'Make',
                        'Model',
                        'Color',
                        'Transmission',
                        'WheelType', 
                        'Size',
                        'VNST',
                        'Axle',
                        'Mileage',
                        'Cylinder']

#replace ? data with Nan so we can properly aggregate
kick_data[continuous_features] = kick_data[continuous_features].replace(to_replace='?', value = -9999999)
kick_data[ordinal_features] = kick_data[ordinal_features].replace(to_replace='?', value = -9999999)
kick_data[categorical_features] = kick_data[categorical_features].replace(to_replace='?', value = 'UNKNOWN')

kick_data[continuous_features] = kick_data[continuous_features].astype(np.float64)
kick_data[ordinal_features] = kick_data[ordinal_features].astype(np.int64)

kick_data[continuous_features] = kick_data[continuous_features].replace(to_replace=-9999999, value = np.nan)
kick_data[ordinal_features] = kick_data[ordinal_features].replace(to_replace=-9999999, value = np.nan)

In [5]:
#Review of overall data quality
def nullcounts(ser):
    return ser.isnull().sum()

def detail_describe(frame, func=['count', nullcounts, 'mean', 'std', 'min', 'median', 'max'],
                    numeric_only=False, **kwargs):
    return frame.agg(func, **kwargs)

detail_describe(kick_data).transpose()

,count,nullcounts,mean,std,min,median,max
IsBadBuy,72983,0,0.122988,0.328425,0,0,1
Auction,72983,0,NaN,NaN,ADESA,NaN,OTHER
VehicleAge,72983,0,4.17664,1.71221,0,4,9
Make,72983,0,NaN,NaN,ACURA,NaN,VOLVO
Model,72983,0,NaN,NaN,1500 RAM PICKUP 2WD,NaN,ZEPHYR 3.0L V6 SFI
Color,72983,0,NaN,NaN,BEIGE,NaN,YELLOW
Transmission,72983,0,NaN,NaN,AUTO,NaN,UNKNOWN
WheelType,72983,0,NaN,NaN,Alloy,NaN,UNKNOWN
VehOdo,72983,0,71500,14578.9,4825,73361,115717
Size,72983,0,NaN,NaN,COMPACT,NaN,VAN


In [6]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer

#create a max/min scaler
kick_scaled_imputed = kick_data

# we want to predict the X and y data as follows:
if 'IsBadBuy' in kick_scaled_imputed:
    y = kick_scaled_imputed['IsBadBuy'].values # get the labels we want
    del kick_scaled_imputed['IsBadBuy'] # get rid of the class label
    X = kick_scaled_imputed.values # use everything else to predict!

#split with stratification on y
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=99, 
                                                    stratify=y)

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(missing_values=np.nan, strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])


#numeric_features = kick_scaled_imputed.select_dtypes(include=['int64', 'float64']).columns
numeric_features = [kick_scaled_imputed.columns.get_loc(c) 
                    for c in kick_scaled_imputed.select_dtypes(include=['int64', 'float64']).columns 
                    if c in kick_scaled_imputed]

#categorical_features = kick_scaled_imputed.select_dtypes(include=['object']).columns

categorical_features = [kick_scaled_imputed.columns.get_loc(c) 
                        for c in kick_scaled_imputed.select_dtypes(include=['object']).columns
                        if c in kick_scaled_imputed]

preprocessor = ColumnTransformer(
    transformers=[('num', numeric_transformer, numeric_features),
                  ('cat', categorical_transformer, categorical_features)])

In [ ]:
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.model_selection import GridSearchCV

param_grid = {'classifier__n_estimators':range(100,3000,100),
              'classifier__max_samples':range(10,2000,100), 
              'classifier__max_features':range(10,2000,100), 
              'classifier__bootstrap':[True],
              'classifier__bootstrap_features':[True]}

scoring = {'AUC': 'roc_auc', 'F0.5': make_scorer(fbeta_score, beta=0.5)}

classifiers = [BaggingClassifier()]

for classifier in classifiers:
    pipe = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', classifier)])
    pipe.fit(X_train, y_train) 
    predic = pipe.predict(X_test)
    print(classifier)
    print("F0.5 model score: %.3f" % fbeta_score(y_test, predic, 0.5))

bagging = Pipeline(steps=[('preprocessor', preprocessor),
                          ('classifier', classifier)])

grid = GridSearchCV(bagging, param_grid, scoring=scoring, refit='F0.5', 
                    return_train_score=True, verbose=2, n_jobs = -1)

grid.fit(X_train,y_train)
results = grid.cv_results_

BaggingClassifier(base_estimator=None, bootstrap=True, bootstrap_features=False,
                  max_features=1.0, max_samples=1.0, n_estimators=10,
                  n_jobs=None, oob_score=False, random_state=None, verbose=0,
                  warm_start=False)
F0.5 model score: 0.509
Fitting 3 folds for each of 11600 candidates, totalling 34800 fits


/Users/chandlervaughn/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   39.4s
/Users/chandlervaughn/anaconda3/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed: 10.1min
[Parallel(n_jobs=-1)]: Done 341 tasks      | elapsed: 27.8min
[Parallel(n_jobs=-1)]: Done 624 tasks      | elapsed: 51.6min
[Parallel(n_jobs=-1)]: Done 989 tasks      | elapsed: 87.3min
[Parallel(n_jobs=-1)]: Done 1434 tasks      | 

In [25]:
import modin.pandas as pd

ImportError: Please `pip install modin[dask] to install compatible Dask version.

In [203]:
pd.DataFrame(grid.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier__bootstrap,param_classifier__bootstrap_features,param_classifier__max_features,param_classifier__max_samples,param_classifier__n_estimators,params,...,std_train_AUC,split0_test_F0.5,split1_test_F0.5,mean_test_F0.5,std_test_F0.5,rank_test_F0.5,split0_train_F0.5,split1_train_F0.5,mean_train_F0.5,std_train_F0.5
0,2.903264,0.013377,1.572523,0.000103,True,True,10,10,100,"{'classifier__bootstrap': True, 'classifier__b...",...,0.020819,0.000000,0.000000,0.000000,0.000000,25,0.000000,0.000000,0.000000,0.000000
1,10.030147,0.060103,12.386255,0.058130,True,True,10,10,1000,"{'classifier__bootstrap': True, 'classifier__b...",...,0.025496,0.000000,0.000000,0.000000,0.000000,25,0.000000,0.000000,0.000000,0.000000
2,18.475156,0.063040,25.148876,0.189503,True,True,10,10,2000,"{'classifier__bootstrap': True, 'classifier__b...",...,0.005191,0.000000,0.000000,0.000000,0.000000,25,0.000000,0.000000,0.000000,0.000000
3,2.917644,0.007689,1.671816,0.021009,True,True,10,100,100,"{'classifier__bootstrap': True, 'classifier__b...",...,0.052327,0.000000,0.000000,0.000000,0.000000,25,0.000000,0.000000,0.000000,0.000000
4,10.338297,0.096850,12.761657,0.082289,True,True,10,100,1000,"{'classifier__bootstrap': True, 'classifier__b...",...,0.014879,0.000000,0.000000,0.000000,0.000000,25,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,47.101679,0.229457,80.898190,0.004437,False,False,1000,100,1000,"{'classifier__bootstrap': False, 'classifier__...",...,0.003070,0.513986,0.527682,0.520834,0.006848,15,0.526467,0.514358,0.520413,0.006054
104,90.269078,0.439417,150.868271,0.853464,False,False,1000,100,2000,"{'classifier__bootstrap': False, 'classifier__...",...,0.005097,0.518950,0.542720,0.530835,0.011885,12,0.533333,0.530936,0.532135,0.001199
105,22.954481,0.273458,9.507158,0.007446,False,False,1000,1000,100,"{'classifier__bootstrap': False, 'classifier__...",...,0.004890,0.534156,0.539229,0.536693,0.002536,11,0.547021,0.540595,0.543808,0.003213
106,177.150130,0.213176,55.002929,1.591215,False,False,1000,1000,1000,"{'classifier__bootstrap': False, 'classifier__...",...,0.002823,0.538111,0.541431,0.539771,0.001660,6,0.550168,0.540867,0.545518,0.004651


In [204]:
grid.best_params_

{'classifier__bootstrap': True,
 'classifier__bootstrap_features': True,
 'classifier__max_features': 1000,
 'classifier__max_samples': 1000,
 'classifier__n_estimators': 1000}

In [ ]:
#Build Final Model with Final Hyperparameters

In [24]:
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

param_grid_final = {'classifier__n_estimators':range(100,2000,10),
                    'classifier__max_samples':[0.1,1,0.1],
                    'classifier__max_features':[0.1,1,0.1], 
                    'classifier__bootstrap':[True, False],
                    'classifier__bootstrap_features':[True, False]}

scoring = {'AUC': 'roc_auc', 'F0.5': make_scorer(fbeta_score, beta=0.5)}

classifiers = [BaggingClassifier()]

for classifier in classifiers:
    pipe = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', classifier)])
    pipe.fit(X_train, y_train) 
    predic = pipe.predict(X_test)
    print(classifier)
    print("F0.5 model score: %.3f" % fbeta_score(y_test, predic, 0.5))

bagging = Pipeline(steps=[('preprocessor', preprocessor),
                          ('classifier', classifier)])

grid_final = RandomizedSearchCV(bagging, param_grid_final, scoring=scoring, refit='F0.5', 
                    return_train_score=True, verbose=2, n_jobs = -1, cv = 5, n_iter = 10)

grid_final.fit(X_train,y_train)
results_final = grid_final.cv_results_
pd.DataFrame(results_final)

BaggingClassifier(base_estimator=None, bootstrap=True, bootstrap_features=False,
                  max_features=1.0, max_samples=1.0, n_estimators=10,
                  n_jobs=None, oob_score=False, random_state=None, verbose=0,
                  warm_start=False)
F0.5 model score: 0.512
Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
/Users/chandlervaughn/anaconda3/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  7.2min


KeyboardInterrupt: 

In [21]:
r = pd.DataFrame(results_final)
r = r[r['param_classifier__bootstrap'] == True]
r = r[r['param_classifier__bootstrap_features'] == True]
r = r[r['param_classifier__max_features'] == 1000]
r = r[r['param_classifier__max_samples'] == 1000]
r = r[r['param_classifier__n_estimators'] == 1000]
r = r.reset_index()
r

,index,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_classifier__n_estimators,param_classifier__max_samples,param_classifier__max_features,param_classifier__bootstrap_features,param_classifier__bootstrap,...,split1_test_F0.5,split2_test_F0.5,mean_test_F0.5,std_test_F0.5,rank_test_F0.5,split0_train_F0.5,split1_train_F0.5,split2_train_F0.5,mean_train_F0.5,std_train_F0.5


In [ ]:
predic = grid.predict(X_test)

In [ ]:
bagging = Pipeline(steps=[('preprocessor', preprocessor),
                          ('classifier', classifier)])

In [ ]:
bagging.fit(X_train, y_train)

In [178]:
for classifier in classifiers:
    pipe = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', classifier)])
    pipe.fit(X_train, y_train)   
    print(classifier)
    print("model score: %.3f" % pipe.score(X_test, y_test))
    




ValueError: Specifying the columns using strings is only supported for pandas DataFrames

In [ ]:




grid = GridSearchCV(bagging, param_grid, scoring=scoring, refit='AUC', return_train_score=True, verbose=2, n_jobs = -1,cv=5)

grid.fit(X_train,y_train)
results = grid.cv_results_

In [163]:
#should be int
ordinal_features = ['VehicleAge', 
                    'BYRNO', 
                    'IsOnlineSale']

#Imputing after split
imputer = SimpleImputer(missing_values=np.nan, strategy="most_frequent")

#Impute Train
df = pd.DataFrame(X_train)
df.rename(columns={ 0:'Auction', 
                    1:'VehicleAge', 
                    2:'Make',
                    3:'Model', 
                    4:'Color', 
                    5:'Transmission', 
                    6:'WheelType', 
                    7:'VehOdo',
                    8:'Size',
                    9:'MMRAcquisitionAuctionAveragePrice',
                    10:'MMRAcquisitionRetailAveragePrice',
                    11:'BYRNO',
                    12:'VNST',
                    13:'VehBCost',
                    14:'IsOnlineSale',
                    15:'WarrantyCost',
                    16:'Cylinder',
                    17:'Axle',
                    18:'Mileage'}, inplace=True)

df[categorical_features] = imputer.fit_transform(df[categorical_features])
df[ordinal_features] = imputer.fit_transform(df[ordinal_features])

#one hot encoding
tmp = pd.concat([pd.get_dummies(df[col],prefix=col) for col in categorical_features], axis=1)
one_hot = pd.concat((df, tmp), axis=1)

#remove the categoricals
for col in categorical_features:
    del one_hot[col]

X_train = one_hot.values

In [164]:
# Impute Test
df = pd.DataFrame(X_test)
df.rename(columns={ 0:'Auction', 
                    1:'VehicleAge', 
                    2:'Make',
                    3:'Model', 
                    4:'Color', 
                    5:'Transmission', 
                    6:'WheelType', 
                    7:'VehOdo',
                    8:'Size',
                    9:'MMRAcquisitionAuctionAveragePrice',
                    10:'MMRAcquisitionRetailAveragePrice',
                    11:'BYRNO',
                    12:'VNST',
                    13:'VehBCost',
                    14:'IsOnlineSale',
                    15:'WarrantyCost',
                    16:'Cylinder',
                    17:'Axle',
                    18:'Mileage'}, inplace=True)

df[categorical_features] = imputer.fit_transform(df[categorical_features])
df[ordinal_features] = imputer.fit_transform(df[ordinal_features])

#one hot encoding
tmp = pd.concat([pd.get_dummies(df[col],prefix=col) for col in categorical_features], axis=1)
one_hot = pd.concat((df, tmp), axis=1)

#remove the categoricals
for col in categorical_features:
    del one_hot[col]

X_test = one_hot.values

In [166]:
pd.DataFrame(X_test)

,0,1,2,3,4,5,6,7,8,9,...,16583,16584,16585,16586,16587,16588,16589,16590,16591,16592
0,84053,2152,1,0,0,0,1,0,0,0,...,1,0,1,0,1,0,1,0,1,0
1,77773,2208,1,0,0,0,0,0,0,0,...,1,0,1,0,1,0,1,0,1,0
2,73169,1389,0,0,0,0,0,1,0,0,...,0,1,1,0,1,0,1,0,1,0
3,71218,1974,1,0,0,0,1,0,0,0,...,1,0,1,0,1,0,1,0,1,0
4,75306,1974,0,1,0,0,1,0,0,0,...,1,0,1,0,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14592,60045,1118,0,1,0,0,0,0,0,0,...,1,0,1,0,1,0,1,0,1,0
14593,39644,1301,1,0,0,0,0,0,0,0,...,1,0,1,0,1,0,1,0,0,1
14594,75553,569,0,1,0,0,0,0,0,1,...,1,0,1,0,1,0,1,0,1,0
14595,64503,533,0,0,0,0,0,0,0,0,...,0,1,1,0,1,0,1,0,1,0


In [161]:
# Scaling after split and after one_hot
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 45)

ValueError: operands could not be broadcast together with shapes (14597,894) (1160,) (14597,894) 

In [ ]:
import pandas as pd
import numpy as np
import h2o
try:
    #clean slate, in case cluster was already running
    h2o.connect()
    h2o.shutdown(prompt=False)
    h2o.remove_all()
except:
    print("couldnt connect, no h2o found which is fine")

h2o.init(max_mem_size = "12G")  #specify max number of bytes. uses all cores by default.

from h2o.estimators.glm import H2OGeneralizedLinearEstimator
h2o_frame = h2o.H2OFrame(kick_data)

In [ ]:
#Set variables and columns
Price_y = 'MMRAcquisitionAuctionAveragePrice'
Price_x = h2o_frame.col_names
try:
    Price_x.remove('state') #duplicate column
    Price_x.remove('zip') #duplicate column
    Price_x.remove('SubModel')
except:
    print("Columns not found. Drop has already been performed.")

Price_x.remove(Price_y)
Price_x.remove('PurchDate')
Price_x.remove('VehYear')
Price_x.remove('VNZIP1')
Price_x.remove('WheelTypeID')
Price_x.remove('Nationality')
Price_x.remove('model_short') 
Price_x.remove('TopThreeAmericanName') 
Price_x.remove('Trim')
Price_x.remove('IsBadBuy')
Price_x.remove('Transmission')
Price_x.remove('Auction')
Price_x.remove('Luxury')
Price_x.remove('Make')
Price_x.remove('VNST')
#Price_x.remove("MMRAcquisitionRetailAveragePrice")

print("Response = " + str(Price_y))
print("Predictors = " + str(Price_x))

In [ ]:
#Outline datasets
train,test,valid = h2o_frame.split_frame(ratios=[.7, .15])
print('Total Dataset: ' + str(h2o_frame.shape))
print('Train: ' + str(train.shape))
print('Validate: ' + str(valid.shape))
print('Test: ' + str(test.shape))

# Scale & Impute Values on Train

In [ ]:
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()
valid[y] = valid[y].asfactor()

train[continuous_features] = train[continuous_features].scale(center=True, scale=True)